# Processing raw XMM-Newton Pointed data

This tutorial will teach you how to use DAXA to process raw XMM-Newton data into a science ready state using one line of Python code (or several lines, if you wish to have more control over the settings for each step). **This relies on there being an initialised (either manually before launching Python, or in your bash profile/rc) backend installation of the XMM Science Analysis System (SAS), including accessible calibration files** - DAXA will check for such an installation, and will not allow processing to start without it.

**All DAXA processing steps will parallelise as much as possible - processes running on different ObsIDs/instruments/sub-exposures will be run simultaneously (if cores are available)**

## Import Statements

In [14]:
from daxa.mission import XMMPointed
from daxa.archive import Archive
from daxa.process.simple import full_process_xmm
from daxa.process.xmm.setup import cif_build, odf_ingest
from daxa.process.xmm.assemble import emchain, epchain

## An Archive to be processed

Every processing function implemented in DAXA takes an Archive instance as its first argument; if you don't already know what that is then you should go back and read the following tutorials:

* [Creating a DAXA archive](archives.html) - This explains how to create an archive, load an existing archive, and the various properties and features of DAXA archives.
* [Using DAXA missions](missions.html) - Here we explain what DAXA mission classes are and how to use them to select only the data you need.

Here we create an archive of XMM observations of the galaxy cluster Abell 907:

In [5]:
xm = XMMPointed()
xm.filter_on_name('A907')
arch = Archive('Abell907', xm)

/Users/dt237/code/DAXA/daxa/mission/xmm.py:83: UserWarning: 140 of the 17689 observations located for this mission have been removed due to NaN RA or Dec values
  self._fetch_obs_info()


## One-line solution

Though we provide individual functions that wrap the various steps required to reduce and prepare XMM data, and they can be used separately for greater control over the configuration parameters, we also include a one-line solution which executes the processing steps with default configuration.

We believe that the default parameters are adequate for most use cases, and this allows for users unfamiliar with the intricacies of XMM data to easily start working with it. Executing the following will automatically generate cleaned event lists for MOS1, MOS2, PN, RGS1, and RGS2 (if they were selected during mission creation), as well as images and exposure maps for MOS1, MOS2, and PN:

In [6]:
full_process_xmm(arch)

XMM-Newton Pointed - Generating calibration files: 100%|████████████████████████| 3/3 [00:19<00:00,  6.51s/it]
XMM-Newton Pointed - Generating ODF summary files: 100%|████████████████████████| 3/3 [00:03<00:00,  1.25s/it]
XMM-Newton Pointed - Assembling PN and PN-OOT event lists: 100%|████████████████| 3/3 [03:37<00:00, 72.36s/it]
XMM-Newton Pointed - Assembling MOS event lists: 100%|██████████████████████████| 6/6 [01:22<00:00, 13.70s/it]
XMM-Newton Pointed - Finding PN/MOS soft-proton flares: 100%|███████████████████| 9/9 [00:17<00:00,  1.93s/it]
XMM-Newton Pointed - Generating cleaned PN/MOS event lists: 100%|███████████████| 8/8 [00:03<00:00,  2.62it/s]
XMM-Newton Pointed - Generating final PN/MOS event lists: 100%|████████████████| 8/8 [00:00<00:00, 120.67it/s]
Generating products of type(s) expmap: 100%|████████████████████████████████████| 8/8 [00:22<00:00,  2.84s/it]


## Breaking down the XMM processing steps

### Setup steps

These functions set up the necessary environment/files for the further processing/reduction of XMM data.

#### Building calibration files (cif_build)

The [cif_build function](../../daxa.process.xmm.html#daxa.process.xmm.setup.cif_build) function is a wrapper for the XMM SAS tool of the same name - it assembles a CIF, which points other SAS tools to the current calibration files required for whatever XMM instrument they are working on.

Only the `analysis_date` parameter of this function affects the files produced by this function (other things can be controlled, such as the number of cores or timeout) - this date determines which calibration files are selected for the generated CIF, and the default is the current date - another date can be passed as a Python datetime object.

**All other processing steps depend on this one**

In [12]:
help(cif_build)

Help on function cif_build in module daxa.process.xmm.setup:

cif_build(obs_archive: daxa.archive.base.Archive, num_cores: int = 9, disable_progress: bool = False, analysis_date: Union[str, datetime.datetime] = 'now', timeout: astropy.units.quantity.Quantity = None) -> Tuple[dict, dict, dict, str, int, bool, astropy.units.quantity.Quantity]
    A DAXA Python interface for the SAS cifbuild command, used to generate calibration files for XMM observations
    prior to processing. The observation date is supplied by the XMM mission instance(s), and is the date when the
    observation was started (as acquired from the XSA).
    
    :param Archive obs_archive: An Archive instance containing XMM mission instances for which observation calibration
        files should be generated. This function will fail if no XMM missions are present in the archive.
    :param int num_cores: The number of cores to use, default is set to 90% of available.
    :param bool disable_progress: Setting this to tr

#### Summarising the available data files (odf_ingest)

The [odf_ingest function](../../daxa.process.xmm.html#daxa.process.xmm.setup.odf_ingest) function simply examines the observation data file (ODF) directory, and determines the instruments (including observing modes and sub-exposures) that have data present - it then creates a SAS summary file.

Our implementation of this function wraps the original SAS tool, and then adds a second step - this parses the SAS summary file and extracts the information that is relevant to whether we can use a particular instrument (and sub-exposure of an instrument) for astrophysics. This is what populates the XMM entry in the `observation_summaries` property of the archive class.

**All subsequent steps depend on this one - and observation_summaries will have no XMM entry until this is run**

In [13]:
help(odf_ingest)

Help on function odf_ingest in module daxa.process.xmm.setup:

odf_ingest(obs_archive: daxa.archive.base.Archive, num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    This function runs the SAS odfingest task, which creates a summary of the raw data available in the ODF
    directory, and is used by many SAS processing tasks.
    
    :param Archive obs_archive: An Archive instance containing XMM mission instances for which observation summary
        files should be generated. This function will fail if no XMM missions are present in the archive.
    :param int num_cores: The number of cores to use, default is set to 90% of available.
    :param bool disable_progress: Setting this to true will turn off the SAS generation progress bar.
    :param Quantity timeout: The amount of time each individual process is allowed to run for, the default is None.
        Please note that this is not a timeout for the entire odf_ingest process, but 

### EPIC Cameras (CCD)

#### Assembling whole-camera MOS initial event lists (emchain)

The [emchain function](../../daxa.process.xmm.html#daxa.process.xmm.assemble.emchain) is what assembles the raw, separate-CCD-level, event lists and files into a single raw events list for a whole XMM MOS camera exposure. If there are **multiple sub-exposures** which DAXA has identified as being usable, then a separate raw event list is created for each of them - they are combined in a later processing step.

We have implemented this method so that MOS1 and MOS2 (and sub-exposures, if present) data are processed in parallel for a given observation (and all observations are processed in parallel as well).

Only the `process_unscheduled` argument passed to emchain can change the files produced, as it controls whether unscheduled sub-exposures should be processed or not - the default is True.

In [15]:
help(emchain)

Help on function emchain in module daxa.process.xmm.assemble:

emchain(obs_archive: daxa.archive.base.Archive, process_unscheduled: bool = True, num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    This function runs the emchain SAS process on XMM missions in the passed archive, which assembles the
    MOS-specific ODFs into combined photon event lists - rather than the per CCD files that existed before. The
    emchain manual can be found here (https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emchain.pdf) and
    gives detailed explanations of the process.
    
    The DAXA wrapper does not allow emchain to automatically loop through all the sub-exposures for a given
    ObsID-MOSX combo, but rather creates a separate process call for each of them. This allows for greater
    parallelisation (if on a system with a significant core count), but also allows the same level of granularity
    in the logging of processing of diffe

#### Assembling whole-camera PN initial event lists (epchain)

The [epchain function](../../daxa.process.xmm.html#daxa.process.xmm.assemble.epchain) serves much the same purpose as emchain, but for PN camera data. There is only one PN camera, but the presence of multiple sub-exposures is still possible. This function generates raw, whole-camera, event lists just like emchain, but also creates whole-camera out-of-time (OOT) event lists, which attempts to identify event lists that were detected during a CCD readout (this can leave a characteristic streak in the direction of readout for bright sources in PN observations).

In [16]:
help(epchain)

Help on function epchain in module daxa.process.xmm.assemble:

epchain(obs_archive: daxa.archive.base.Archive, process_unscheduled: bool = True, num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    This function runs the epchain SAS process on XMM missions in the passed archive, which assembles the
    PN-specific ODFs into combined photon event lists - rather than the per CCD files that existed before. A run of
    epchain for out of time (OOT) events is also performed as part of this function call. The epchain manual can be
    found here (https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epchain.pdf) and gives detailed
    explanations of the process.
    
    Per the advice of the SAS epchain manual, the OOT event list epchain call is performed first, and its intermediate
    files are saved and then used for the normal call to epchain.
    
    :param Archive obs_archive: An Archive instance containing XMM mission instanc

### Reflection Grating Spectrometer (RGS)

## 